# Stock Sentiment Analysis using News Headlines

In [537]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.set_option("display.max_columns",None)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
import warnings
warnings.filterwarnings("ignore")


In [538]:
data=pd.read_csv("Data.csv",encoding = "ISO-8859-1")

In [539]:
data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,Derby raise a glass to Strupar's debut double,"Southgate strikes, Leeds pay the penalty",Hammers hand Robson a youthful lesson,Saints party like it's 1999,Wear wolves have turned into lambs,Stump mike catches testy Gough's taunt,Langer escapes to hit 167,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,Hopkins 'furious' at Foster's lack of Hannibal...,Has Cubie killed fees?,A tale of two tails,I say what I like and I like what I say,"Elbows, Eyes and Nipples",Task force to assess risk of asteroid collision,How I found myself at last,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite
2,2000-01-05,0,Coventry caught on counter by Flo,United's rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,McGrath puts India out of their misery,Blair Witch bandwagon rolls on,Pele turns up heat on Ferguson,Party divided over Kohl slush fund scandal,Manchester United (England),Women in record South Pole walk,Vasco da Gama (Brazil),South Melbourne (Australia),Necaxa (Mexico),Real Madrid (Spain),Raja Casablanca (Morocco),Corinthians (Brazil),Tony's pet project,Al Nassr (Saudi Arabia),Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,2000-01-06,1,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,"Auntie back in fashion, even if the dress look...",Shoaib appeal goes to the top,Hussain hurt by 'shambles' but lays blame on e...,England's decade of disasters,Revenge is sweet for jubilant Cronje,"Our choice, not theirs",Profile of former US Nazi Party officer Willia...,New evidence shows record of war crimes suspec...,The rise of the supernerds,Written on the body,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for...,Christmas glitches,"Upending a table, Chopping a line and Scoring ...","Scientific evidence 'unreliable', defence claims",Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake-up of failing NHS,Lessons of law's hard heart
4,2000-01-07,1,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers: are you all whingers?,Hollywood Beyond,Ashes and diamonds,Whingers - a formidable minority,Alan Parker - part two,"Thuggery, Toxins and Ties",Met faces fresh attack on race crime,Everton fans top racist 'league of shame',"Our breasts, ourselves",Russia's new boss has an extremely strange his...,Always and forever,Most everywhere: UDIs,Most wanted: Chloe lunettes,

In [540]:
data.shape

(4101, 27)

In [541]:
data["Date"].describe()

count           4101
unique          4101
top       2004-12-20
freq               1
Name: Date, dtype: object

In [542]:
data[data["Date"]<"2015-01-01"].shape

(3723, 27)

In [543]:
# split train and test based on date column
train = data[data['Date'] < '20150101']
test= data[data['Date'] >= '20150101']


In [544]:
print(len(train))
print(len(test))

3975
126


In [545]:
train.replace("[^A-Za-z]"," ",regex=True,inplace=True)
test.replace("[^A-Za-z]"," ",regex=True,inplace=True)

In [546]:
train.drop(columns=["Date"],inplace=True)
test.drop(columns=["Date"],inplace=True)

In [547]:
train.head(2)

,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,0,A hindrance to operations extracts from the...,Scorecard,Hughes instant hit buoys Blues,Jack gets his skates on at ice cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,Derby raise a glass to Strupar s debut double,Southgate strikes Leeds pay the penalty,Hammers hand Robson a youthful lesson,Saints party like it s,Wear wolves have turned into lambs,Stump mike catches testy Gough s taunt,Langer escapes to hit,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl s successor drawn into scandal,The difference between men and women,Sara Denver nurse turned solicitor,Diana s landmine crusade put Tories in a panic,Yeltsin s resignation caught opposition flat f...,Russian roulette,Sold out,Recovering a title
1,0,Scorecard,The best lake scene,Leader German sleaze inquiry,Cheerio boyo,The main recommendations,Has Cubie killed fees,Has Cubie killed fees,Has Cubie killed fees,Hopkins furious at Foster s lack of Hannibal...,Has Cubie killed fees,A tale of two tails,I say what I like and I like what I say,Elbows Eyes and Nipples,Task force to assess risk of asteroid collision,How I found myself at last,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man s extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn t know without the ...,Millennium bug fails to bite


In [548]:
top=train.iloc[:,1:]

In [549]:
top.iloc[0]

Top1     A  hindrance to operations   extracts from the...
Top2                                             Scorecard
Top3                       Hughes  instant hit buoys Blues
Top4              Jack gets his skates on at ice cold Alex
Top5                Chaos as Maracana builds up for United
Top6     Depleted Leicester prevail as Elliott spoils E...
Top7                      Hungry Spurs sense rich pickings
Top8                     Gunners so wide of an easy target
Top9         Derby raise a glass to Strupar s debut double
Top10             Southgate strikes  Leeds pay the penalty
Top11                Hammers hand Robson a youthful lesson
Top12                          Saints party like it s     
Top13                   Wear wolves have turned into lambs
Top14               Stump mike catches testy Gough s taunt
Top15                            Langer escapes to hit    
Top16             Flintoff injury piles on woe for England
Top17    Hunters threaten Jospin with new battle of the.

In [550]:
headings=[]
for i in range(0,len(top)):
    headings.append(" ".join(top.iloc[i,:].astype(str)).lower())

In [551]:
headings[0]

'a  hindrance to operations   extracts from the leaked reports scorecard hughes  instant hit buoys blues jack gets his skates on at ice cold alex chaos as maracana builds up for united depleted leicester prevail as elliott spoils everton s party hungry spurs sense rich pickings gunners so wide of an easy target derby raise a glass to strupar s debut double southgate strikes  leeds pay the penalty hammers hand robson a youthful lesson saints party like it s      wear wolves have turned into lambs stump mike catches testy gough s taunt langer escapes to hit     flintoff injury piles on woe for england hunters threaten jospin with new battle of the somme kohl s successor drawn into scandal the difference between men and women sara denver  nurse turned solicitor diana s landmine crusade put tories in a panic yeltsin s resignation caught opposition flat footed russian roulette sold out recovering a title'

In [552]:
# implement bag of words
count_vector=CountVectorizer(ngram_range=(2,2))
train_data_set=count_vector.fit_transform(headings)

In [553]:
rf_classifier=RandomForestClassifier()
rf_classifier.fit(train_data_set,train["Label"])

RandomForestClassifier()

In [554]:
test.head

<bound method NDFrame.head of       Label                                               Top1  \
3975      0  Saudi Arabia breaks off ties with Iran   Amid ...   
3976      1  Dutch government backs strong encryption  cond...   
3977      0  China shuts down stock market after    min    ...   
3978      0  Reports of sexual assaults on women across Eur...   
3979      0  El Chapo  Being Taken to Same Prison He Escape...   
...     ...                                                ...   
4096      0  Barclays and RBS shares suspended from trading...   
4097      1        Scientists To Australia  If You Want To ...   
4098      1                   Explosion At Airport In Istanbul   
4099      1  Jamaica proposes marijuana dispensers for tour...   
4100      1  A     year old woman in Mexico City finally re...   

                                                   Top2  \
3975  Exclusive  UK Government urged to reveal its r...   
3976  ISIS Militants Strap Bomb on   Year Old Boy an...   


In [555]:
test.reset_index(inplace=True)

In [556]:
top=test.iloc[:,2:]

In [557]:
top.head(1)

,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,Saudi Arabia breaks off ties with Iran Amid ...,Exclusive UK Government urged to reveal its r...,China halts trading as stocks fall,Earthquake measuring magnitude strikes India,A virus linked to shrinking newborns brains is...,New photos of SpaceX booster show sooty but u...,Indian consulate in Afghanistan under attack b...,Iran calls off Major Hajj,Denmark responds to Swedish border checks with...,David Cameron criticised for turning blind ey...,Denmark introduces temporary controls along Ge...,Israel Retaliates to Hezbollah bombing at Leba...,China Aims to Build Nuclear Power Plants in...,According to obtained documents after rape an...,Western Australian rooftop solar systems produ...,Iranian leaders website posts controversial im...,After Bahrain Sudan expels Iranian ambassador...,Saudi Arabia to cut off all air traffic with I...,Hezbollah sets off large border bomb targeting...,Iranian diplomats given hours to leave Saud...,BBC News Migrant crisis Sweden border checks...,Teachers claim the politics of austerity is le...,The Aftermath of a Muslim Cleric s Execution i...,Iran arrested for attack on Saudi embassy,Oil prices jump over pct on Saudi Arabia Ir...


In [558]:
test.index

RangeIndex(start=0, stop=126, step=1)

In [559]:
test_headings=[]
for i in test.index:
    test_headings.append(" ".join(test.iloc[i].astype(str)).lower())

In [560]:
len(test_headings)

126

In [561]:
len(test)

126

In [562]:
test_data=count_vector.transform(test_headings)
prediction=rf_classifier.predict(test_data)

In [563]:
matrix=confusion_matrix(test["Label"],prediction)
accuracy=accuracy_score(test["Label"],prediction)
report=classification_report(test["Label"],prediction)
print(matrix)
print(accuracy)
print(report)

[[12 44]
 [ 6 64]]
0.6031746031746031
              precision    recall  f1-score   support

           0       0.67      0.21      0.32        56
           1       0.59      0.91      0.72        70

    accuracy                           0.60       126
   macro avg       0.63      0.56      0.52       126
weighted avg       0.63      0.60      0.54       126



In [564]:
naive=MultinomialNB()
naive.fit(train_data_set,train["Label"])

MultinomialNB()

In [565]:
naive_prediction=naive.predict(test_data)

In [566]:
matrix=confusion_matrix(test["Label"],naive_prediction)
accuracy=accuracy_score(test["Label"],naive_prediction)
report=classification_report(test["Label"],naive_prediction)
print(matrix)
print(accuracy)
print(report)

[[ 8 48]
 [10 60]]
0.5396825396825397
              precision    recall  f1-score   support

           0       0.44      0.14      0.22        56
           1       0.56      0.86      0.67        70

    accuracy                           0.54       126
   macro avg       0.50      0.50      0.45       126
weighted avg       0.51      0.54      0.47       126

